In [39]:
# 理解装饰器其实就是高阶函数

from functools import wraps

def log(func):
    @wraps(func)
    def wrapper(*args, **kw):
        print("Flag")
        log_string = func.__name__ + ' was called' 
        print(log_string)
        return func(*args, **kw)
    return wrapper

@log
def foo():
    print('2024--3-09')
    # pass
    return

def now():
    print('2024--3-09')
    return
    


print("foo() results:")
foo()
print('\n')

print("log(foo)() results:")
log(foo)()
print('\n')

print('log(foo()) results:')
log(foo())
print('\n')

try:
    log(foo())()
except AttributeError:
    print("AttributeError")


print('\n')
print('now() results:')
now()
print('\n')

print('log(now)() results:')
log(now)()
print('\n')

print('log(log(now))() results:')
log(log(now))()
print('\n')

print('log(now()) results:')
log(now())
print('\n')

try:
    log(now())()
except AttributeError:
    print("AttributeError")

foo() results:
Flag
foo was called
2024--3-09


log(foo)() results:
Flag
foo was called
Flag
foo was called
2024--3-09


log(foo()) results:
Flag
foo was called
2024--3-09


Flag
foo was called
2024--3-09
Flag
AttributeError


now() results:
2024--3-09


log(now)() results:
Flag
now was called
2024--3-09


log(log(now))() results:
Flag
now was called
Flag
now was called
2024--3-09


log(now()) results:
2024--3-09


2024--3-09
Flag
AttributeError


In [45]:
from functools import wraps

"""
这里，是为了提醒 return func(*args, **kw)
比如foo()没有传参数，为什么在`with_loggint`中还要`return func(*args, **kw)`
因为有时候foo(x)是有参数的，此时`return func()`会引发TypeError
所以此处最好都写成`return func(*args, **kw)`
"""

def logit(func):
    @wraps(func)
    def with_logging(*args, **kw):
        log_string = func.__name__ + ' was called' 
        print(log_string)
        return func()
        # return func(*args, **kw)
    return with_logging

@logit
def foo():
    pass

foo()
print('-------------------')

@logit
def addition_func(x):
   """Do some math."""
   return x + x
 
try :
    result = addition_func(4)
except TypeError:
    print("此处引发TypeErrof是因为在with_logging()函数内部，`return func()`没有传递参数导致的， \n"+
          "所以应该养成习惯，被装饰函数无论有没有参数，都应写为retun func(*args, **kw)")

foo was called
-------------------
addition_func was called
此处错误是因为在with_logging()函数内部，`return func()`没有传递参数导致的， 
所以应该养成习惯，被装饰函数无论有没有参数，都应写为retun func(*args, **kw)


### 带参数的装饰器

In [20]:
from functools import wraps

def logit(logfile = 'out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kw):
            log_string = func.__name__+" was called."
            print(log_string) 
            with open(logfile, 'a') as opened_file:
                opened_file.write(log_string + '\n')
            return func(*args, **kw)
        return wrapped_function
    return logging_decorator

@logit() #<<<<<<<<<<<<<<<<<一定注意此处添加()
def foo():
    pass

foo()

@logit('bar.log')
def bar():
    pass


bar()

foo was called.
bar was called.


### 类装饰器

有时你只想打日志到一个文件。而有时你想把引起你注意的问题发送到一个email，同时也保留日志，留个记录。这是一个使用继承的场景，但目前为止我们只看到过用来构建装饰器的函数。

幸运的是，类也可以用来构建装饰器。那我们现在以一个类而不是一个函数的方式，来重新构建logit。

In [23]:
from functools import wraps

class logit():
    def __init__(self, logfile = 'out.log'):
        self.logfile = logfile

    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kw):
            log_string = func.__name__ + ' was called.' 
            print(log_string)
            
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
                
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kw)
        return wrapped_function
    
    def notify(self):
        # logit只打日志，不做别的
        pass
    
@logit()
def foo():
    pass

foo()

foo was called.


In [26]:
# 现在，我们给 logit 创建子类，来添加 email 的功能(虽然 email 这个话题不会在这里展开)。
class email_logit(logit):
    def __init__(self, email = 'guanghui-hua@qq.com', *args, **kw):
        self.email = email
        super(email_logit, self).__init__(*args, **kw)
        
    def notify(self):
        # 发送一封email到self.email
        # 这里就不做实现了
        print("Has sent an email to admin!")
        pass

@email_logit()
def bar():
    pass

bar()
# 从现在起，@email_logit 将会和 @logit 产生同样的效果，但是在打日志的基础上，还会多发送一封邮件给管理员

bar was called.
Has sent an email to admin!
